# Channel Flow - private kli

Results published: preliminary study 


### Preliminaries

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
//#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `KOmegaSolver` and additional namespaces:

In [ ]:
using BoSSS.Application.KOmega;
using BoSSS.Application.RANSCommon;
using BoSSS.Solution.NSECommon;
using NUnit.Framework;

Initialization of the Workflow management; there is the project name `ChannelFlowKOmega` which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("ChannelFlowKOmega");

Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


In [ ]:
//var myBatch = ExecutionQueues[0];
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ]


## Grid Creation - Channel

In [ ]:
// channel length
double l = 0.1;
// channel half-width
double h = 1;

(int cellsx, int cellsy)[] Resolutions = new [] { (20, 40) };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];

bool periodicBC = true;

In [ ]:
for(int i = 0; i < Resolutions.Length; i++) {
    var Res = Resolutions[i];
    string GridName = $"ChannelFlowKOmega_{Res.cellsx}x{Res.cellsy}";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        var _xNodes = GenericBlas.Linspace(-l, l, Res.cellsx);
        var _yNodes = GenericBlas.Linspace(-h, h, Res.cellsy);

        var grd = Grid2D.Cartesian2DGrid(_xNodes,
                                         _yNodes,
                                         CellType.Square_Linear,
                                         periodicX: periodicBC
                                         );
        grd.Name = GridName;

        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;

            if (Math.Abs(X.y - (-h)) < 1.0e-8)
                // bottom
                ret = IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString();

            if (Math.Abs(X.y - (h)) < 1.0e-8)
                // top
                ret = IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString();

            if (!periodicBC) {
                if (Math.Abs(X.x - (-l)) < 1.0e-8)
                    // left
                    ret = IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString();

                if (Math.Abs(X.x - (l)) < 1.0e-8)
                    // right
                    ret = IncompressibleBcType.Pressure_Outlet.ToString() + "_" + TurbulenceModelBcType.Neumann.ToString();
            }

            return ret;
        });     
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\ChannelFlowKOmega'.
Grid already found in database - identifid by name ChannelFlowKOmega_20x40


## Initial Values

In [ ]:
var VelXInit = new Formula(
    "VelX",
    false,
    "using ilPSP.Utils; " + 
    "double VelX(double[] X) { " + 
    "    double vmax = 21.262;" + 
    "    double posWall = 0.0;" + 
    "    double height = 1.0;" + 
    "    return ((vmax - posWall) * (1 - (X[1] / height) * (X[1] / height))) + posWall; " + 
    "}");

In [ ]:
var VelYInit = new Formula(
    "VelY",
    false,
    "using ilPSP.Utils; " + 
    "double VelY(double[] X) { " + 
    "    double vmax = 21.262;" + 
    "    double posWall = 0.0;" + 
    "    double height = 1.0;" + 
    "    return 0.0; " + 
    "}");

In [ ]:
var PressureInit = new Formula(
    "Pressure",
    false,
    "using ilPSP.Utils; " + 
    "double Pressure(double[] X) { " + 
    "    double viscosity= 7.540114783971117e-05;" + 
    "    double kWall = 6e-1;" + 
    "    double omegaTildeWall = Math.Log(6e2);" + 
    "    return (viscosity + kWall / omegaTildeWall) * (2.5 - X[0]); " + 
    "}");

In [ ]:
var kPolyInit = new Formula(
    "kPoly",
    false,
    "using ilPSP.Utils; " + 
    "double kPoly(double[] X) { " + 
    "    double turbulentIntensity = 0.8;" + 
    "    double Umean= 1.0;" + 
    "    double kIn = 3 / 2.0 * Math.Pow(Umean * turbulentIntensity, 2);" + 
    "    double kWall = 6e-1;" + 
    "    double height = 1.0;" + 
    "    return ((kIn - kWall) * (1 - (X[1] / height) * (X[1] / height))) + kWall; " + 
    "}");

In [ ]:
var omegaTildeInit = new Formula(
    "omegaTilde",
    false,
    "using ilPSP.Utils; " + 
    "double omegaTilde(double[] X) { " + 
    "    double turbulentIntensity = 0.8;" + 
    "    double Umean= 1.0;" + 
    "    double kIn = 3 / 2.0 * Math.Pow(Umean * turbulentIntensity, 2);" + 
    "    double L = 0.4;" +
    "    double omegaTildeIn = Math.Log(Math.Sqrt(kIn) / L);" + 
    "    double omegaTildeWall = Math.Log(6e2);" + 
    "    double height = 1.0;" + 
    "    return ((omegaTildeIn - omegaTildeWall) * (1 - (X[1] / height) * (X[1] / height))) + omegaTildeWall; " + 
    "}");

## Setup of control objects for all solver runs

In [ ]:
List<KOmegaControl> Controls = new List<KOmegaControl>();
Controls.Clear();

int[] DegreeS = new int[] { 1 };

foreach(int k in DegreeS) {
foreach(var grd in Grids.Take(1)) {

    long J = grd.NumberOfCells;
    string JobName = $"CFKO_J{J}k{k}";
    Console.WriteLine("Case: " + JobName);

    var C = new KOmegaControl();
    
    C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    

    C.NonLinearSolver.MaxSolverIterations = 100;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.NonLinearSolver.ConvergenceCriterion = 1E-10;
    C.LinearSolver.MaxSolverIterations = 10;
    C.LinearSolver.MinSolverIterations = 3;
    C.LinearSolver.ConvergenceCriterion = 1E-10;
    C.dt = 1e20; // pseudo-timestep
    // C.dt = 100;
    C.dtMax = C.dt;
    C.dtMin = C.dt;
    // C.dtMax = 0.001;
    // C.dtMin = 100.0;
    C.Timestepper_Scheme = KOmegaControl.TimesteppingScheme.ImplicitEuler;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    // C.LinearSolver.SolverCode = LinearSolverCode.classic_mumps;
    // C.LinearSolver.SolverCode = LinearSolverCode.exp_gmres_levelpmg;
    // C.LinearSolver.SolverCode = LinearSolverCode.exp_direct_lapack;
    C.LinearSolver.SolverCode = LinearSolverCode.classic_pardiso;
    // C.LinearSolver.SolverCode = LinearSolverCode.exp_softgmres;

    C.AdaptiveMeshRefinement = false;
    C.RefinementLevel = 10;

    C.UseArtificialDiffusion = false;

    C.Bodyforces = false;
    //C.WhichWall = "Wall_Cylinder";

    //Debugging and Solver Analysis
    C.OperatorMatrixAnalysis = false;
    C.AnalysisLevel = 2;
    C.SetInitialConditions = true;
    C.SetInitialPressure = true;
    C.SetParamsAnalyticalSol = false;
    C.ComputeL2Error = false;
    C.GravitySource = false;
    C.GravityX = (X, t) => 0;
    C.GravityY = (X, t) => 0;

    //Physical Params
    C.Stokes = false;
    C.FixedStreamwisePeriodicBC = periodicBC;
    C.Viscosity = 7.540114783971117e-05;

    C.useJacobianForOperatorMatrix = true;

    //Penalties
    C.ViscousPenaltyScaling = 1;
    C.Penalty2 = 1;
    C.Penalty1[0] = 0.0;
    C.Penalty1[1] = 0.0;
    C.PresPenalty2 = 1;
    C.PresPenalty1[0] = 0.0;
    C.PresPenalty1[1] = 0.0;

    // Set Initial Conditions
    C.InitialValues.Add(VariableNames.Velocity_d(0), VelXInit);
    C.InitialValues.Add(VariableNames.Velocity_d(1), new Formula("(X) => 0.0", false));

    C.InitialValues.Add("k", kPolyInit);
    C.InitialValues.Add("omega_tilde", omegaTildeInit);

    // Set Boundary Conditions
    C.AddBoundaryValue(IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "VelocityX", new Formula("(X) => 0.0", false));
    C.AddBoundaryValue(IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "VelocityY", new Formula("(X) => 0.0", false));
    C.AddBoundaryValue(IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "k", kPolyInit); // kWall
    C.AddBoundaryValue(IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "omega_tilde", omegaTildeInit); // omegaWall

    if (!C.FixedStreamwisePeriodicBC) {
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), VariableNames.Velocity_d(0), VelXInit);
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), VariableNames.Velocity_d(1), new Formula("(X) => 0.0", false));
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "k", kPolyInit); // kIn // kWall
                         
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "omega_tilde",  omegaTildeInit); // omegaIn // omegaWall
                          
        // //C.AddBoundaryCondition("Velocity_inlet",VariableNames.Pressure, Pressurefunction);
        // C.AddBoundaryValue("Pressure_Outlet_neumann");
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), VariableNames.Pressure, new Formula("(X) => 1.0", false));
        C.AddBoundaryValue(IncompressibleBcType.Pressure_Outlet.ToString() + "_" + TurbulenceModelBcType.Neumann.ToString(), VariableNames.Pressure, new Formula("(X) => 0.0", false));
        // C.AddBoundaryValue("Pressure_Outlet_neumann", "k", X =>
        //                    kPolynomial (X,0)
        //                    // kIn
        //                    // kWall
        //                    );
        // C.AddBoundaryValue("Pressure_Outlet_neumann", "omega", X =>
        //                    omegaPolynomial (X,0)
        //                    // omegaIn
        //                    // omegaWall
        //                    );
        C.AddBoundaryValue(IncompressibleBcType.Pressure_Outlet.ToString() + "_" + TurbulenceModelBcType.Neumann.ToString());

        // C.AddBoundaryValue("Wall_top");
        // C.AddBoundaryValue("Wall_bottom");
        // C.AddBoundaryValue("Velocity_inlet");
        // C.AddBoundaryValue("Pressure_Outlet_neumann");
    }
    
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}

Case: CFKO_J741k1


In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal");
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal");
    }
}

## Launch Jobs

In [ ]:
Controls.Count

1

In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Deploying job CFKO_J741k1 ... 
Deploying executables and additional files ...
Deployment directory: D:\local\binaries\ChannelFlowKOmega-KOmegaSolver2022Feb25_140836
copied 45 files.
   written file: control.obj
deployment finished.
Starting mini batch processor in external process...
Started mini batch processor on local machine, process id is 25196.
started.



In [ ]:
wmg.AllJobs

#0: CFKO_J741k1: PendingInExecutionQueue (MiniBatchProcessor client  LocalPC @D:\local\binaries)	CFKO_J741k1: InProgress (MiniBatchProcessor client  LocalPC @D:\local\binaries)


In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

Error: System.OperationCanceledException: Command :SubmitCode: // wait for all jobs to finish (up to 5 days, chec ... cancelled.

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

Starting external console ...
(You may close the new window at any time, the job will continue.)


In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()

In [ ]:
//wmg.Sessions.Pick(0).Timesteps